In [1]:
import pandas as pd
import geopandas as gpd
import altair as alt

In [2]:
start = '2017-01-01'
end = '2023-12-31'

## Mittlerer Strompreis der letzen sieben Jahre

* Jahre 2017 bis 2023
* Pro Gemeinde
* Wenn für ein oder mehrere Jahre kein Preis verfügbar ist, nehmen wir den Schnitt der verbleibenden Jahre

In [3]:
## Mittlerer Marktpreis aus 1_marktpreis_bfe.ipynb
%store -r mittel_0
%store -r mittel_4_6
%store -r mittel_8
marktpreis =  mittel_4_6
marktpreis

9.633233571428573

Daten der Elektrizitätswerke der letzten 7 Jahre von [pvtarif.ch](https://www.vese.ch/pvtarif/)

Als Vergleichspreis nehmen wir `energy1` für die Jahre 2017 bis 2023 sowie die Vergütung für den herkunftsnachweis.

In [4]:
ew = pd.read_csv('daten/ew.csv')
ew['strompreis'] = ew['energyAuto1'] + ew['ecoAuto1']
ew['Datum'] = pd.to_datetime(ew['year'], format='%y')

ew = ew[['Datum', 'nrElcom', 'strompreis', 'nomEw', 'year']]
ew = ew.query('Datum >= "'+start+'" & Datum <= "'+end+'"')
ew.query('strompreis.isna()')

,Datum,nrElcom,strompreis,nomEw,year
775,2019-01-01,769,NaN,Technisches Werk Elektrizität Braunau,19
782,2019-01-01,767,NaN,Technische Werke der Gemeinde Tobel-Tägerschen,19
1691,2023-01-01,88,NaN,Elektra-Genossenschaft Bellikon-Hausen,23
2171,2022-01-01,88,NaN,Elektra-Genossenschaft Bellikon-Hausen,22
2477,2020-01-01,185,NaN,ELEKTRA Seeberg - Grasswil - Riedtwil,20
2522,2020-01-01,162,NaN,Elektra Mümliswil-Ramiswil,20
2536,2020-01-01,147,NaN,Elektra Itingen,20
2682,2020-01-01,730,NaN,Städtische Elektrizitätsversorgung Laufenburg,20
2758,2020-01-01,769,NaN,Technisches Werk Elektrizität Braunau,20
2929,2021-01-01,524,NaN,Energieversorgung Aegerten EVA,21


Pro Elektrizitätswerk den mittleren Preis der Jahre 2017 bis 2023 berechnen:

In [5]:
ew1 = ew.groupby('nrElcom').agg({
    'strompreis': 'mean', ## Mittelwert über sieben Jahre
    'nomEw': 'first',
}, skipna=True).reset_index()
ew1.query('strompreis.isna()')

,nrElcom,strompreis,nomEw
50,88,NaN,Elektra-Genossenschaft Bellikon-Hausen


Elektrizitäswerke nach Gemeinden von [pvtarif.ch](https://www.vese.ch/pvtarif/):

In [6]:
gd = pd.read_csv('daten/gd.csv')
gd = gd[['nrElcom', 'idofs', 'Gemeinde']].rename({'idofs': 'ID'}, axis=1)
gd

,nrElcom,ID,Gemeinde
0,486,1,Aeugst am Albis
1,486,2,Affoltern am Albis
2,486,3,Bonstetten
3,486,4,Hausen am Albis
4,486,5,Hedingen
...,...,...,...
2308,36,6806,Vendlincourt
2309,36,6807,Basse-Allaine
2310,36,6808,Clos du Doubs
2311,36,6809,Haute-Ajoie


Für jedes Jahr, Elektrizitätswerk und jede Gemeinde eine eigene Zeile. Einzelne Gemeinden haben mehrere Anbieter. Manche Anbieter haben nicht für jedes Jahr Daten.

In [7]:
merged = pd.merge(ew, gd, on='nrElcom', how='outer')
merged.sort_values(['Gemeinde', 'nomEw', 'year']).to_excel('calc/ew_gd.xlsx', index=False)
merged.query('strompreis.isna()')

,Datum,nrElcom,strompreis,nomEw,year,ID,Gemeinde
5219,2021-01-01,524,NaN,Energieversorgung Aegerten EVA,21.0,731,Aegerten
5315,2020-01-01,185,NaN,ELEKTRA Seeberg - Grasswil - Riedtwil,20.0,988,Seeberg
8474,2020-01-01,162,NaN,Elektra Mümliswil-Ramiswil,20.0,2428,Mümliswil-Ramiswil
11697,2020-01-01,730,NaN,Städtische Elektrizitätsversorgung Laufenburg,20.0,4170,Laufenburg
12093,2019-01-01,769,NaN,Technisches Werk Elektrizität Braunau,19.0,4723,Braunau
...,...,...,...,...,...,...,...
14995,NaT,255,NaN,NaN,NaN,6054,Binn
14996,NaT,680,NaN,NaN,NaN,6142,Trient
14997,NaT,543,NaN,NaN,NaN,6291,Saas-Grund
14998,NaT,543,NaN,NaN,NaN,6293,Stalden (VS)


Wir gehen davon aus, dass der Strom in jeder Gemeinde an den Anbieter verkauft wird, der den höchsten Preis bietet:

In [8]:
d1 = merged.groupby(['ID', 'year']).agg({
    'strompreis': 'max', # Höchster Preis pro Gemeinde und Jahr
    'Gemeinde': 'first'
}, skipna=True).reset_index()
d1.sort_values(['Gemeinde', 'year']).to_excel('calc/ew_gd_2.xlsx', index=False)
d1.query('Gemeinde == "Davos"')

,ID,year,strompreis,Gemeinde
7931,3851,17.0,8.59,Davos
7932,3851,18.0,9.37,Davos
7933,3851,19.0,11.19,Davos
7934,3851,20.0,10.48,Davos
7935,3851,21.0,11.40,Davos
7936,3851,22.0,12.98,Davos
7937,3851,23.0,18.10,Davos


In [9]:
einwohner = pd.read_csv("daten/einwohner.csv")
einwohner.head()

,ID,Gemeinde,Einwohner
0,1,Aeugst am Albis,1981
1,2,Affoltern am Albis,12303
2,3,Bonstetten,5572
3,4,Hausen am Albis,3751
4,5,Hedingen,3778


In [10]:
d1a = pd.merge(d1, einwohner[['ID', 'Einwohner']], on='ID', how='left')
d1a.to_csv('calc/preis_gemeinde_jahr.csv', index=False)
d1a

,ID,year,strompreis,Gemeinde,Einwohner
0,1,17.0,6.23,Aeugst am Albis,1981
1,1,18.0,5.53,Aeugst am Albis,1981
2,1,19.0,5.53,Aeugst am Albis,1981
3,1,20.0,7.29,Aeugst am Albis,1981
4,1,21.0,7.30,Aeugst am Albis,1981
...,...,...,...,...,...
14217,6810,19.0,9.50,La Baroche,1142
14218,6810,20.0,7.68,La Baroche,1142
14219,6810,21.0,14.03,La Baroche,1142
14220,6810,22.0,30.00,La Baroche,1142


Mittlerer Preis pro Gemeinde über die Jahre 2017 bis 2023:

In [11]:
d2 = d1a.groupby('ID').agg({
    'strompreis': 'mean', # Mittelwert über sieben Jahre
    'Gemeinde': 'first',
    'Einwohner': 'first'
}, skipna=True).reset_index() # Fehlen Jahre, wird der Mittelwert der restlichen Jahre berechnet
d2.query('Gemeinde == "Davos"')

,ID,strompreis,Gemeinde,Einwohner
1167,3851,11.73,Davos,10862


Nach Einwohner gewichteter Mittelwert.   
Einwohnerzahlen von https://www.bfs.admin.ch/bfs/de/home/statistiken/regionalstatistik/regionale-portraets-kennzahlen/gemeinden.html

In [12]:
d3 = d2.copy()
d3.to_csv('calc/preis_gemeinde.csv', index=False)
d3

,ID,strompreis,Gemeinde,Einwohner
0,1,7.551429,Aeugst am Albis,1981
1,2,7.551429,Affoltern am Albis,12303
2,3,7.551429,Bonstetten,5572
3,4,7.551429,Hausen am Albis,3751
4,5,7.551429,Hedingen,3778
...,...,...,...,...
2103,6806,12.214286,Vendlincourt,559
2104,6807,12.214286,Basse-Allaine,1238
2105,6808,12.214286,Clos du Doubs,1263
2106,6809,12.214286,Haute-Ajoie,1085


In [13]:
d3['weight'] = d3['Einwohner'] / d3['Einwohner'].sum()
d3

,ID,strompreis,Gemeinde,Einwohner,weight
0,1,7.551429,Aeugst am Albis,1981,0.000232
1,2,7.551429,Affoltern am Albis,12303,0.001443
2,3,7.551429,Bonstetten,5572,0.000654
3,4,7.551429,Hausen am Albis,3751,0.000440
4,5,7.551429,Hedingen,3778,0.000443
...,...,...,...,...,...
2103,6806,12.214286,Vendlincourt,559,0.000066
2104,6807,12.214286,Basse-Allaine,1238,0.000145
2105,6808,12.214286,Clos du Doubs,1263,0.000148
2106,6809,12.214286,Haute-Ajoie,1085,0.000127


In [14]:
(d3.strompreis * d3.weight).sum()

10.412160752303988